In [2]:
import pandas as pd
import numpy as np
import os
import torch
import matplotlib.pyplot as plt

In [1]:
DIR = '/home/jarlene/Desktop/stock/csv'

In [6]:
def rename(path):
    names = ['open', 'high', 'low', 'close', 'adj.close', 'volume', 'dividends', 'stock.splits', 'capital.gains']
    data = pd.read_csv(path)
    cols = data.columns.to_list()
    cols[0] = 'date'
    colss =['date']
    for c in cols[1:]:
        ss = c.split('.')
        if (len(ss) == 1) or (not ss[-1].isnumeric() and len(ss) > 1):
            ss.append("0")
        ss[-1] = "_"+ names[int(ss[-1])]
        colss.append('.'.join(ss[:-1]) + ss[-1])
    data.columns = colss
    data = data.drop([0,1])
    data.to_csv(path, sep='\t', index=False)

    
def preprocess(path, num_attris = 9):
    data = pd.read_csv(path, sep='\t')
    trade_date_lenth = num_attris
    cols_len = len(data.columns) - 1
    trade_num = int(cols_len/trade_date_lenth)
    all_cols = data.columns.to_list()
    dataes = [data.iloc[:, 1+i*trade_date_lenth:trade_date_lenth *
                        (i+1) + 1] for i in range(trade_num)]
    cols = [all_cols[1+i*trade_date_lenth:trade_date_lenth *
                        (i+1) + 1] for i in range(trade_num)]
    for i in range(len(dataes)):
        d = dataes[i]
        d['date'] = data['date']
        d.dropna(inplace=True)
        d.columns = cols[i] +['date']
        for col in cols[i][:5]:
            d[col] = d[col].pct_change()
        

    res = [i.to_numpy() for i in dataes]
    torch.save(res, path.replace('.csv', '.pt'))

def merge(files, save_path):
    res = []
    for f in files:
        data = torch.load(f)
        res.extend(data)
    
    torch.save(res, save_path)

In [32]:
for f in os.listdir(DIR):
    f_path = os.path.join(DIR, f)
    if f.endswith('.csv') and os.path.isfile(f_path):
        preprocess(f_path)

In [3]:
files = []
for f in os.listdir(DIR):
    f_path = os.path.join(DIR, f)
    if f.endswith('.pt') and os.path.isfile(f_path):
        files.append(f_path)

In [4]:
files

['/home/jarlene/Desktop/stock/csv/USMF_25000.pt',
 '/home/jarlene/Desktop/stock/csv/TSX.pt',
 '/home/jarlene/Desktop/stock/csv/USMF_15000.pt',
 '/home/jarlene/Desktop/stock/csv/USMF_20000.pt',
 '/home/jarlene/Desktop/stock/csv/OTCBB-004.pt',
 '/home/jarlene/Desktop/stock/csv/LSE.pt',
 '/home/jarlene/Desktop/stock/csv/SGX.pt',
 '/home/jarlene/Desktop/stock/csv/USMF_0.pt',
 '/home/jarlene/Desktop/stock/csv/MGEX.pt',
 '/home/jarlene/Desktop/stock/csv/CFE.pt',
 '/home/jarlene/Desktop/stock/csv/EUREX.pt',
 '/home/jarlene/Desktop/stock/csv/ASX.pt',
 '/home/jarlene/Desktop/stock/csv/NYBOT.pt',
 '/home/jarlene/Desktop/stock/csv/USMF_5000.pt',
 '/home/jarlene/Desktop/stock/csv/TSXV.pt',
 '/home/jarlene/Desktop/stock/csv/LIFFE.pt',
 '/home/jarlene/Desktop/stock/csv/USMF_10000.pt',
 '/home/jarlene/Desktop/stock/csv/AMEX.pt',
 '/home/jarlene/Desktop/stock/csv/FOREX.pt',
 '/home/jarlene/Desktop/stock/csv/NASDAQ.pt',
 '/home/jarlene/Desktop/stock/csv/INDEX.pt']

In [7]:
merge(files, os.path.join(DIR,'res.pt'))

: 

: 